In [1]:
import numpy as np 
import pandas as pd
import pandas3js as pjs
print(pjs.__version__)

0.1.6


# Empty GUI

In [ ]:
gui, geometry, opts = pjs.views.create_gui(camera_position=[0,0,-10])
gui

In [ ]:
geometry.add_object(pjs.models.Sphere(id=1,color='blue',label='S'))
geometry.add_object(pjs.models.Box(id=2,position=[1,1,1],
                                   color='green',label='B'))
geometry

# Simple GUI Options and Callback

In [ ]:
def callback(geometry, options):
    for obj in geometry:
        obj.label = options['label']
        obj.color = options['color']
        obj.transparency = options['opaque']
    return
  
geometry = pjs.models.GeometricCollection()
geometry.add_object(pjs.models.Sphere(id=1,label='S'))
geometry.add_object(pjs.models.Box(id=2,position=[2,1,1],
                                   label='B'))

gui, geometry, opts = pjs.views.create_gui(geometry, callback,
                                     opts_choice={'label':['A','B']},
                                     opts_color={'color':'blue'},
                                     opts_range={'opaque':np.arange(1.0, 0.1, -0.1)})
gui

In [ ]:
opts

# Object Types

In [ ]:
def change_func(geometry,options):
    # change id each time to generate new object
    df = pd.DataFrame({
        'id':[np.random.randint(10**3)],
        'color':[options['Color']],
        'transparency':.8,
        'otype':['pandas3js.models.'+options['Object']],
         'position':[(options['x'],0,0)],
         'visible':[options['Visible']]})
    if 'Icosahedron' in options['Object']:
        df['detail'] = 1
    if 'Triclinic' in options['Object']:
        pjs.utils.tuple_to_df(df,'b',(0.5,1,0))
        pjs.utils.tuple_to_df(df,'c',(0,.5,1))
    geometry.change_by_df(df,otype_column='otype')

gui, gcollect, opts = pjs.views.create_gui(callback=change_func,
       opts_choice={'Visible':[True,False],
                'Object':['Sphere','Box','Line', 
                'TriclinicWire','TriclinicSolid',
                'Octahedron','Icosahedron','Circle',
                'Plane']},
       opts_color={'Color':'red'},opts_range={'x':range(0,5)},
       height=200,width=200,add_labels=False,
       view=(3,-3,-2,4),otype_column='otype',
        orthographic=True)

gui

# Options Initialisation and Layout

In [ ]:
data = {'1':{'id':[0,1,2],
             'position':[(0,1,0),(1,1,0),(2,0,1)],
             'name':['A','B','C'],
             'charge':[1,-1,.5],
             'c1':'blue','c2':'red'},
        '2':{'id':[0,1,2],
             'position':[(0,1,0),(.5,0,.5),(1,1,1)],
             'name':['A','B','C'],
             'charge':[1,-1,.5],
             'c1':'blue','c2':'red'}}

def callback(geometry,options):
    """ converts the data and options 
        to a dataframe of geometry 
    """
    cdata = data[options['Configuration']]
    df = pd.DataFrame(cdata)[['id','position']]
    df['otype'] = 'pandas3js.models.Sphere'
    ctype = options.get('Color','c1')
    df['color'] = cdata[ctype]
    df['radius'] = options.get('Radius',1)
    df['label'] = cdata['name']
    df['other_info'] = ['a sphere<br>charge = %.2f $C.m^{-3}$' % q 
                        for q in cdata['charge']]
    geometry.change_by_df(df,otype_column='otype')

gui, gcollect, opts = pjs.views.create_gui(callback=callback,
            height=200,width=200,camera_position=[.5,0,-4],
            show_object_info=True,otype_column='otype',
            opts_choice={'Color':['c1','c2']},
            opts_range={'Configuration':['1','2'],
                       'Radius':np.arange(0.1,2,.1).round(2)},
            layout=[('Configuration',['Configuration']),
                    ('Geometry',['Color','Radius'])],
            initial_values={'Color':'c2','Radius':.4})
gui

In [ ]:
trait_df = gcollect.trait_df()
trait_df

In [ ]:
trait_df.transparency = 0.1
gcollect.change_by_df(trait_df.loc[0:1],
                      remove_missing=True,
                      otype_column='otype')

# Use with jsonextended to visualise simulations

In [ ]:
# pip install jsonextended
from jsonextended import edict, utils, _example_data_folder
from jsonextended.complex_parsers import crystal_dft

In [ ]:
datapath = utils.get_data_path('data.crystal.out',_example_data_folder)
parser = crystal_dft.CrystalDFTParser()
data = parser.read_file(datapath)

In [ ]:
data = edict.combine_lists(data,['id','x/a','y/b','z/c','atomic_number'],'ixyzn')
geometry = pjs.models.GeometricCollection()

# create bounding box
ldict = data['initial']['primitive']['lattice_parameters']
a_vec, b_vec, c_vec = pjs.atom.lattice_from_params(
     *[ldict[s] for s in ('a','b','c','alpha','beta','gamma')])
bbox = pjs.models.TriclinicWire(id=-1,a=a_vec,b=b_vec,c=c_vec,color='black')  
geometry.add_object(bbox)

# create atoms
for d in data['initial']['primitive']['geometry']['ixyzn'].values():
    x,y,z = d['x/a']*a_vec+d['y/b']*b_vec+d['z/c']*c_vec
    #atom = pjs.models.Sphere(id=d['id'],position=(x,y,z))
    atom = pjs.models.Icosahedron(id=d['id'],position=(x,y,z),detail=1)
    atom.color = atom.label_color = pjs.atom.map_atoms(d['atomic_number'],'color')
    atom.radius = pjs.atom.map_atoms(d['atomic_number'],'RCov')
    atom.label = pjs.atom.map_atoms(d['atomic_number'],'Symbol')
    geometry.add_object(atom)    

# repeat unit cell
pjs.atom.repeat_cell(geometry,a_vec)
pjs.atom.repeat_cell(geometry,b_vec)

def callback(geometry, options):
    for obj in geometry:
        if options['Hide Fe'] and obj.label == 'Fe':
            obj.visible = False
        else:
            obj.visible = True

gui, geometry, opts = pjs.views.create_gui(geometry,callback,
                                     opts_choice={'Hide Fe':[False,True]},
                                     orthographic=True,add_labels=True)
gui